In [1]:
import pandas as pd
from datetime import datetime

## Data Manipulation

In [2]:
#INPUT DATA
customer={
"_id" : ["600c1806289947de938c68ea"],
"name" : ["Ram Mohan"],
"age" : [32],
"gender" : ["male"],
"dateJoined": ['2019-09-30T09:18:29.044Z'],
"state" : ["MH"],
"country": ["IN"]
}

farms={
"_id" : ["uTMEtfc77M8eLdX7E"],
"customer" : ["600c1806289947de938c68ea"],
"name" : ["farm_tomato"],
"crop" : ["tomato"],
"deviceId": ["7HGRXY"],
"lat" :[20.632189],
"long": [74.8961]
}

Device={
"_id" : ["uTMEtfc77M8eLdX7E"],
"deviceId" : ["7HGRXY"],
"TC" : [24],
"HUM" : [84],
"datetime": ['2021-07-30T09:18:29.044Z'],
"LW" : [0],
"PLV2": [0]
}

#Convert Dict to DataFrame
customer=pd.DataFrame(customer)
farm=pd.DataFrame(farms)
device=pd.DataFrame(Device)

#Convert datetime to DateTime format
device['datetime']=device['datetime'].astype('datetime64')

# Hourly data in Device
device['hourly']=device['datetime'].apply(lambda x: x.replace(minute=0, second=0,microsecond=0))

# Joining Tables
df=pd.merge(customer,farm,left_on='_id',right_on='customer')
df=pd.merge(df,device,left_on='_id_y',right_on='_id')

#Rearrangign Columns
df=df[['customer','_id_y','crop','deviceId_y', 'TC', 'HUM', 'LW', 'PLV2','datetime', 'hourly']]
df=df.rename(columns={'deviceId_y':'device_id','_id_y':'farm_id','customer':'customer_id'})

## Function to get_data

In [3]:
#Function (get_data) to extract data for a given crop, input date range and set of sensor values.
def get_data(crop,start_date,end_date,sensor_values):
    df1=df[df['crop']==crop]
    df1=df1[(df1['datetime']>=start_date) & (df1['datetime']<=end_date)]
    df1=df1[sensor_values]
    return df1

#Function (get_data_by_hour) to extract hourly data for a given crop, input date range and set of sensor values.
def get_data_by_hour(crop,start_date,end_date,sensor_values):
    df1=df[df['crop']==crop]
    df1=df1[(df1['datetime']>=start_date) & (df1['datetime']<=end_date)]
    df1=df1[sensor_values]
    df1['hourly']=df1['datetime'].apply(lambda x: x.replace(minute=0, second=0,microsecond=0))
    df1=df1.groupby(['customer_id','farm_id','crop','device_id','hourly']).mean() #add cropname and farmname/id,device_id
    return df1


## OUTPUT

In [4]:
#INPUT DATE
start = datetime.strptime('Jan 1 2020', '%b %d %Y').replace(minute=0, second=0,microsecond=0)
end = datetime.strptime('Dec 1 2021', '%b %d %Y').replace(minute=0, second=0,microsecond=0)

get_data('tomato',start,end,['TC','HUM'])

,TC,HUM
0,24,84
